In [ ]:
# python
import os, sys, pickle
from itertools import combinations_with_replacement, combinations, product
from collections import OrderedDict

# stats
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.api import OLS


# plot
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

# neural networks
import torch, torch.utils.model_zoo  # required to load nets
from torchvision.models.feature_extraction import (
    get_graph_node_names,
    create_feature_extractor,
)

# analysis code
from lib.transforms import VisualPriorRepresentation
from lib.functions_second_analysis import *
from lib.functions_scripting import *
    # DATASET_NAMES,
    # SCALE_NAMES,
    # STUDY_NAMES,
    # BEHAVIOUR_NAMES,
    # MODEL_NAMES,
    # load_integration,
    # load_ibcorr,
    # load_pvalues,
    # load_ratings,
    # studyname2datasetname,
    # NETS_SEMANTIC,
    # NETS_2D,
    # NETS_3D,
    # NETS_ALL,
#)

PATH_IMAGES = "../images and ratings/imageversions_256"
PATH_RATINGS = "../images and ratings/ratings"

# !! correlations, invert sign for integration
PATH_INTEGRATION = "../data csv/integration"
PATH_INTEGRATION_MAXPOOL = "../data csv/integration maxpool"
PATH_INTEGRATION_AVERAGE = "../data csv/integration average"

PATH_IBCORR = "../data csv/ibcorr"
PATH_IBCORR_AVERAGE = "../data csv/ibcorr average"
PATH_IBCORR_MAXPOOL = "../data csv/ibcorr maxpool"

PATH_RESULTS = "../results"
PATH_PLOTS = "../plots"

_models = MODEL_NAMES
_study = "short presentation"
_dataset = "places1"
_scale = "scale4"

beauty_ratings = load_ratings(PATH_RATINGS)

In [ ]:
# opt to subselect models
models_to_remove = set(
    ["edge_occlusion", "inpainting", "keypoints", "normal", "jigsaw"]
)
_models = list(set(MODEL_NAMES).difference(models_to_remove))

In [ ]:
# load & subselect unblocked data
_blocking = None
_layers = slice(None)

dfi = load_integration(PATH_INTEGRATION)
dfibc = load_ibcorr(PATH_IBCORR)

dfi = dfi.sort_index().loc[(_models, _dataset, _scale, slice(None), _layers)]
dfi = dfi.droplevel(["dataset","scale"])

dfibc = dfibc.sort_index().loc[(_models, _study, _scale, _layers)]
dfibc = dfibc.droplevel(["study","scale"])

In [ ]:
# load & subselect averaged data
_blocking = "average"
_layers = slice(None)

dfi = load_integration(PATH_INTEGRATION_AVERAGE)
dfibc = load_ibcorr(PATH_IBCORR_AVERAGE)

dfi = dfi.sort_index().loc[(_models, _dataset, _scale, slice(None), _layers)]
dfi = dfi.droplevel(["dataset", "scale"])

dfibc = dfibc.sort_index().loc[(_models, _study, _scale, _layers)]
dfibc = dfibc.droplevel(["study", "scale"])

In [ ]:
# load & subselect maxpooled data
_blocking = "maxpool"
_layers = slice(6, 14)

dfi = load_integration(PATH_INTEGRATION_MAXPOOL)
dfibc = load_ibcorr(PATH_IBCORR_MAXPOOL)

dfi = dfi.sort_index().loc[(_models, _dataset, _scale, slice(None), _layers)]
dfi = dfi.droplevel(["dataset", "scale"])

dfibc = dfibc.sort_index().loc[(_models, _study, _scale, _layers)]
dfibc = dfibc.droplevel(["study", "scale"])

# dfi_m = load_integration(PATH_INTEGRATION_MAXPOOL)
# dfibc_m = load_ibcorr(PATH_IBCORR_MAXPOOL)

# dfi_m = dfi_m.sort_index().loc[(slice(None), _dataset, _scale, slice(None), _layers_blocked)]
# dfi_m = dfi_m.loc[_models]
# dfi_m = dfi_m.droplevel(["dataset","scale"])

# dfibc_m = dfibc_m.sort_index().loc[(slice(None),_study, _scale, _layers_blocked)]
# dfibc_m = dfibc_m.loc[_models]
# dfibc_m = dfibc_m.droplevel(["study","scale"])

In [ ]:
def calculate_predictors_target_rdms(
    path_integration, path_ibcorr, _models, _study, _scale, _layers
):
    _dataset = studyname2datasetname(_study)
    blocking = None

    dfi = load_integration(path_integration)
    dfibc = load_ibcorr(path_ibcorr)

    dfi = dfi.sort_index().loc[(_models, _dataset, _scale, slice(None), _layers)]
    dfi = dfi.droplevel(["dataset", "scale"])

    dfibc = dfibc.sort_index().loc[(_models, _study, _scale, _layers)]
    dfibc = dfibc.droplevel(["study", "scale"])

    # --- target ---
    target_rdm = pd.DataFrame(0, index=_models, columns=_models)
    df = dfibc.unstack("layer")
    for c1, c2 in combinations(_models, 2):
        target_rdm.loc[c1, c2] = (df.loc[c1] - df.loc[c2]).abs().sum()
        target_rdm.loc[c2, c1] = (df.loc[c1] - df.loc[c2]).abs().sum()

    # --- predictor 1 ---
    model_rdm = pd.DataFrame(
        np.full((len(NETS_ALL), len(NETS_ALL)), np.nan),
        columns=NETS_ALL,
        index=NETS_ALL,
    )
    for combi in combinations_with_replacement(NETS_ALL, 2):
        if (
            combi in combinations_with_replacement(NETS_SEMANTIC, 2)
            or combi in combinations_with_replacement(NETS_2D, 2)
            or combi in combinations_with_replacement(NETS_3D, 2)
        ):
            model_rdm.loc[combi] = 1
            model_rdm.loc[tuple(reversed(combi))] = 1
        else:
            model_rdm.loc[combi] = 0
            model_rdm.loc[tuple(reversed(combi))] = 0

    predictor_rdm_1 = pd.DataFrame(0, index=_models, columns=_models)
    for c1, c2 in combinations_with_replacement(_models, 2):
        if c1 in model_rdm.index and c2 in model_rdm.index:
            predictor_rdm_1.loc[c1, c2] = 1 - model_rdm.loc[c1, c2]
            predictor_rdm_1.loc[c2, c1] = 1 - model_rdm.loc[c2, c1]
        else:
            predictor_rdm_1.loc[c1, c2] = 0
            predictor_rdm_1.loc[c2, c1] = 0
    
    # --- predictor 2---
    bestlayers = dfibc.groupby("model").idxmax().ibcorr
    df_bestpredicting_integration = (
        dfi.unstack("img").loc[bestlayers].droplevel("layer").T.droplevel(0)
    )
    predictor_rdm_2 = calculate_rdm(df_bestpredicting_integration, "spearman")

    # ---predictor 3---
    try:
        rdms = {}
        for mo in _models:
            rdm = pd.read_csv(
                os.path.join(
                    PATH_RESULTS,
                    "spatial integration",
                    _scale,
                    _study,
                    mo + ".csv",
                ),
                header=None,
            )
            rdm = rdm.iloc[1:, 1:].reset_index(drop=True).T.reset_index(drop=True).T
            rdms[mo] = rdm
        predictor_rdm_3 = calculate_rdm(flatten_concat_rdms(rdms))
    except Exception as e:
        print("Spatial analysis not available for these parameters, returing emptry predictor 3")
        predictor_rdm_3 = None

    return target_rdm, predictor_rdm_1, predictor_rdm_2, predictor_rdm_3

# Target 01: differences in ib-corr

absoloute difference in correlation in each layer, summed up. <br>
Optional: Normalized with 2 (spearman correlation range) * num_layers 

## version 1: absoloute differences

In [ ]:
df = dfibc.unstack("layer")
rdm_target_absdiff = pd.DataFrame(0, index=_models, columns=_models)

for c1, c2 in combinations(_models, 2):
    # sum of absoloute differences (easier, no correlation of correlated correlation coefficients)
    rdm_target_absdiff.loc[c1, c2] = (df.loc[c1] - df.loc[c2]).abs().sum()
    rdm_target_absdiff.loc[c2, c1] = (df.loc[c1] - df.loc[c2]).abs().sum()

# norming dissimilarity: sum_abs_diff / (num_layers * 2)
sns.heatmap(
    rdm_target_absdiff,
    xticklabels=rdm_target_absdiff.columns,
    yticklabels=rdm_target_absdiff.index,
)
target_rdm = rdm_target_absdiff

## version 2: correlation

In [ ]:
# TODO: add correlate correlations
df = dfibc.unstack("layer")
rdm_target_corrwise = pd.DataFrame(0, index=_models, columns=_models)

for c1, c2 in combinations(_models, 2):
    rdm_target_corrwise.loc[c1, c2] = pearsonr(df.loc[c1], df.loc[c2])[0]
    rdm_target_corrwise.loc[c2, c1] = pearsonr(df.loc[c1], df.loc[c2])[0]

# norming dissimilarity: sum_abs_diff / (num_layers * 2)
sns.heatmap(
    rdm_target_corrwise,
    xticklabels=rdm_target_corrwise.columns,
    yticklabels=rdm_target_corrwise.index,
)
target_rdm = rdm_target_corrwise

# Predictor 1: semantic-2d-3d
Network grouping according to Radek paper <br>
[Finished predictor RDM ](#predictor-rdm-semantic-2d-3d)

In [ ]:
# add network classes to data

# integration
dfi = dfi.loc[NETS_ALL]
dfi["class"] = dfi.reset_index(level="model").model.apply(modelname2class).values
dfi = dfi.set_index("class", append=True)

dfi_m = dfi_m.loc[NETS_ALL]
dfi_m["class"] = dfi_m.reset_index(level="model").model.apply(modelname2class).values
dfi_m = dfi_m.set_index("class", append=True)


# ibcorr
dfibc = dfibc.loc[NETS_ALL]
dfibc["class"] = dfibc.reset_index(level="model").model.apply(modelname2class).values
dfibc = dfibc.set_index("class", append=True)

dfibc_m = dfibc_m.loc[NETS_ALL]
dfibc_m["class"] = (
    dfibc_m.reset_index(level="model").model.apply(modelname2class).values
)
dfibc_m = dfibc_m.set_index("class", append=True)

In [ ]:
# model RDM
model_rdm = pd.DataFrame(
    np.full((len(NETS_ALL), len(NETS_ALL)), np.nan), columns=NETS_ALL, index=NETS_ALL
)

for combi in combinations_with_replacement(NETS_ALL, 2):
    if (
        combi in combinations_with_replacement(NETS_SEMANTIC, 2)
        or combi in combinations_with_replacement(NETS_2D, 2)
        or combi in combinations_with_replacement(NETS_3D, 2)
    ):
        model_rdm.loc[combi] = 1
        model_rdm.loc[tuple(reversed(combi))] = 1
    else:
        model_rdm.loc[combi] = 0
        model_rdm.loc[tuple(reversed(combi))] = 0

sns.heatmap(model_rdm, cmap="viridis")

### variance partitioning

MODEL RDM as target,  steps as predictors <br>
shared variance between any of the models, subsequent added explained variance by each of the steps

In [ ]:
# get beauty predictions for images from best predicting model in each layer for each class
bestnets = (
    dfibc_m.unstack("layer")
    .groupby("class")
    .aggregate(lambda c: c.idxmax()[0])
    .stack("layer")
    .rename({"ibcorr": "net"}, axis=1)
    .reset_index()
    .values.tolist()
)
dfi_m_best = (
    dfi_m.unstack("img")
    .reorder_levels(["class", "layer", "model"])
    .loc[bestnets, :]
    .droplevel("model")
    .stack("img")
    #    .unstack("layer")
)

In [ ]:
dfi_m_best

In [ ]:
ibcorr_from_classmaxpool = (
    dfi_m_best.groupby(["layer", "class"])
    .aggregate(lambda i: pearsonr(-i, beauty_ratings["study1_places1_short.csv"])[0][0])
    .unstack("class")
)
ibcorr_from_classmaxpool.plot()

In [ ]:
# average beauty prediction (i.e. integration) for each image from each class
# since the ib-correlation is the spearman correlation. Just do OLS variance partitioning for now an then talk to daniel about it.
ibcorr_from_classmaxpool = (
    dfi_m_best.groupby(["layer", "class"])
    .aggregate(lambda i: pearsonr(-i, beauty_ratings["study1_places1_short.csv"])[0][0])
    .unstack("class")
)
ibcorr_from_classmaxpool.plot()

ibcorr_from_classavg = (
    dfi_m.groupby(["class", "layer", "img"])
    .mean()
    .groupby(["layer", "class"])
    .aggregate(lambda i: pearsonr(-i, beauty_ratings["study1_places1_short.csv"])[0][0])
    .unstack("class")
)
ibcorr_from_classavg.plot()

In [ ]:
dfi_m_classavg = dfi_m.groupby(["class", "layer", "img"]).mean().unstack("class")
dfi_m_classavg

In [ ]:
beauty_ratings["study1_places1_short.csv"].values.squeeze()

In [ ]:
dfi_m_classavg.loc[:, (slice(None), "2d")]

In [ ]:
r2_all = dfi_m_classavg.groupby(["layer"]).apply(
    lambda X: OLS(beauty_ratings["study1_places1_short.csv"].values.squeeze(), X.values)
    .fit()
    .rsquared
)

In [ ]:
r2_semantic = r2_all - dfi_m_classavg.loc[:, (slice(None), ["2d", "3d"])].groupby(
    ["layer"]
).apply(
    lambda X: OLS(beauty_ratings["study1_places1_short.csv"].values.squeeze(), X.values)
    .fit()
    .rsquared
)

r2_2d = r2_all - dfi_m_classavg.loc[:, (slice(None), ["semantic", "3d"])].groupby(
    ["layer"]
).apply(
    lambda X: OLS(beauty_ratings["study1_places1_short.csv"].values.squeeze(), X.values)
    .fit()
    .rsquared
)


r2_3d = r2_all - dfi_m_classavg.loc[:, (slice(None), ["semantic", "2d"])].groupby(
    ["layer"]
).apply(
    lambda X: OLS(beauty_ratings["study1_places1_short.csv"].values.squeeze(), X.values)
    .fit()
    .rsquared
)

In [ ]:
plt.plot(r2_3d, label="3d")
plt.plot(r2_2d, label="2d")
plt.plot(r2_semantic, label="semantic")
plt.legend()

In [ ]:
# create linear model
# single study & layer
dataset = "places1"
layer_idx = 48

df_icr.loc[dataset, layer_idx]

OLS(Y, X).fit().rsquared

In [ ]:
def do_variance_partitioning():
    # do variance partitioning for one layer
    # i.e. for all unique, shared and full combinations of the three predictors
    # return dataframe with all R2 values
     

## semantic, 2d, 3d along layers

### image activations

### raw integration amount


In [ ]:
# get beauty predictions for images from best predicting model in each layer for each class
bestnets = (
    dfibc_m.unstack("layer")
    .groupby("class")
    .aggregate(lambda c: c.idxmax()[0])
    .stack("layer")
    .rename({"ibcorr": "net"}, axis=1)
    .reset_index()
    .values.tolist()
)
dfi_m_best = (
    dfi_m.unstack("img")
    .reorder_levels(["class", "layer", "model"])
    .loc[bestnets, :]
    .droplevel("model")
    .stack("img")
    #    .unstack("layer")
)

In [ ]:
dfi_m_best

In [ ]:
ibcorr_from_classmaxpool = (
    dfi_m_best.groupby(["layer", "class"])
    .aggregate(lambda i: pearsonr(-i, beauty_ratings["study1_places1_short.csv"])[0][0])
    .unstack("class")
)
ibcorr_from_classmaxpool.plot()

In [ ]:
# average beauty prediction (i.e. integration) for each image from each class
# since the ib-correlation is the spearman correlation. Just do OLS variance partitioning for now an then talk to daniel about it.
ibcorr_from_classmaxpool = (
    dfi_m_best.groupby(["layer", "class"])
    .aggregate(lambda i: pearsonr(-i, beauty_ratings["study1_places1_short.csv"])[0][0])
    .unstack("class")
)
ibcorr_from_classmaxpool.plot()

ibcorr_from_classavg = (
    dfi_m.groupby(["class", "layer", "img"])
    .mean()
    .groupby(["layer", "class"])
    .aggregate(lambda i: pearsonr(-i, beauty_ratings["study1_places1_short.csv"])[0][0])
    .unstack("class")
)
ibcorr_from_classavg.plot()

In [ ]:
dfi_m_classavg = dfi_m.groupby(["class", "layer", "img"]).mean().unstack("class")
dfi_m_classavg

In [ ]:
beauty_ratings["study1_places1_short.csv"].values.squeeze()

In [ ]:
dfi_m_classavg.loc[:, (slice(None), "2d")]

In [ ]:
r2_all = dfi_m_classavg.groupby(["layer"]).apply(
    lambda X: OLS(beauty_ratings["study1_places1_short.csv"].values.squeeze(), X.values)
    .fit()
    .rsquared
)

In [ ]:
r2_semantic = r2_all - dfi_m_classavg.loc[:, (slice(None), ["2d", "3d"])].groupby(
    ["layer"]
).apply(
    lambda X: OLS(beauty_ratings["study1_places1_short.csv"].values.squeeze(), X.values)
    .fit()
    .rsquared
)

r2_2d = r2_all - dfi_m_classavg.loc[:, (slice(None), ["semantic", "3d"])].groupby(
    ["layer"]
).apply(
    lambda X: OLS(beauty_ratings["study1_places1_short.csv"].values.squeeze(), X.values)
    .fit()
    .rsquared
)


r2_3d = r2_all - dfi_m_classavg.loc[:, (slice(None), ["semantic", "2d"])].groupby(
    ["layer"]
).apply(
    lambda X: OLS(beauty_ratings["study1_places1_short.csv"].values.squeeze(), X.values)
    .fit()
    .rsquared
)

In [ ]:
plt.plot(r2_3d, label="3d")
plt.plot(r2_2d, label="2d")
plt.plot(r2_semantic, label="semantic")
plt.legend()

In [ ]:
# create linear model
# single study & layer
dataset = "places1"
layer_idx = 48

df_icr.loc[dataset, layer_idx]

OLS(Y, X).fit().rsquared

In [ ]:
def do_variance_partitioning():
    # do variance partitioning for one layer
    # i.e. for all unique, shared and full combinations of the three predictors
    # return dataframe with all R2 values
     

## semantic, 2d, 3d along layers

### image activations

### raw integration amount


In [ ]:
# average integration of layers
df_int_netavg = df1.groupby("model").mean().transpose()
handles, labels = df_int_netavg.plot().get_legend_handles_labels()

# already order legend by classes
order = [labels.index(netname) for netname in NETS_ALL]
plt.legend(
    [handles[idx] for idx in order],
    [labels[idx] for idx in order],
    loc="center right",
    bbox_to_anchor=(1.5, 0.5),
)

In [ ]:
# average integration, grouped by semantic-2d-3d
colors = (
    len(NETS_SEMANTIC) * ["green"]
    + len(NETS_2D) * ["purple"]
    + len(NETS_3D) * ["orange"]
)

for (netname, int_netavg), color in zip(df_int_netavg.iloc[:, order].items(), colors):
    if netname in NETS_SEMANTIC:
        alpha = 0.7
    else:
        alpha = 0.3
    plt.plot(int_netavg, label=netname, color=color, alpha=alpha)
    plt.legend(loc="center right", bbox_to_anchor=(1.5, 0.5))

#### dev: single layer

In [ ]:
layer_id = 48

# fitler relevant data
layer_df = pd.DataFrame(df.loc[NETS_ALL, "places1", "scale8"][layer_id]).reset_index()
# needed for pivot into wide format
layer_df["img_id"] = layer_df.groupby("model").cumcount()

# pivot
layer_df = layer_df.pivot(columns="model", index="img_id", values=layer_id)

# reorder columns according to semantic-2D-3D nets
layer_df = layer_df[NETS_ALL]

rdm = calculate_rdm(layer_df, correlation_type="spearman")

pearsonr(rdm.values.flatten(), model_rdm.values.flatten())

In [ ]:
pearsonr(rdm.values.flatten(), model_rdm.values.flatten())

In [ ]:
sns.heatmap(rdm, cmap="viridis")

In [ ]:
xdm = rdm[rdm > 0.142].fillna(0)
sns.heatmap(xdm, cmap="viridis")

In [ ]:
xdm = rdm[rdm < 0].fillna(0)
sns.heatmap(xdm, cmap="viridis")

#### all layers

In [ ]:
dfi

In [ ]:
model_correlations = []
model_pvalues = []
# iterate layers
for layer_name, layer_series in df.loc[:, "places1", "scale8"].items():

    # put data back into DataFrame
    layer_df = pd.DataFrame(layer_series).reset_index()

    # needed for pivot into wide format
    layer_df["img_id"] = layer_df.groupby("model").cumcount()

    # pivot
    layer_df = layer_df.pivot(columns="model", index="img_id", values=layer_name)

    # reorder columns according to semantic-2D-3D nets
    layer_df = layer_df[NETS_ALL]

    rdm = calculate_rdm(layer_df, correlation_type="spearman")

    model_correlations.append(
        pearsonr(rdm.values.flatten(), model_rdm.values.flatten())[0]
    )
    model_pvalues.append(pearsonr(rdm.values.flatten(), model_rdm.values.flatten())[1])

In [ ]:
alpha = 0.05

sns.lineplot(data=model_correlations)
plt.suptitle("Similarity in what is integrated")
plt.title("Correlation of taskonomy RDM with model (semantic-2D-3D) RDM")
plt.xlabel("Layer")
plt.ylabel("pearson correlation")


for x, layer_pvalue in enumerate(model_pvalues):
    if layer_pvalue < alpha:
        plt.scatter(x, 0, color="cyan", s=100, marker="o")

### ibcorr differences

## inspect class average integration

In [ ]:
# per class best and worst prediction from individual nets
Y1 = dfibc_m.groupby(["layer", "class"]).agg([min, max])
Y2 = (
    dfi_m.groupby(["class", "layer", "img"])
    .mean()
    .groupby(["layer", "class"])
    .aggregate(lambda i: spearmanr(i, beauty_ratings["study1_places1_short.csv"])[0])
    .unstack("class")
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(12, 3), sharey=True)

modelclass = "semantic"
axes[0].fill_between(
    Y1.loc[(slice(None), modelclass), :].index.get_level_values("layer"),
    Y1.loc[(slice(None), modelclass), (slice(None), "min")].values.flat,
    Y1.loc[(slice(None), modelclass), (slice(None), "max")].values.flat,
)
axes[0].plot(-Y2.loc[:, (slice(None), modelclass)], c="red")

modelclass = "2d"
axes[1].fill_between(
    Y1.loc[(slice(None), modelclass), :].index.get_level_values("layer"),
    Y1.loc[(slice(None), modelclass), (slice(None), "min")].values.flat,
    Y1.loc[(slice(None), modelclass), (slice(None), "max")].values.flat,
)
axes[1].plot(-Y2.loc[:, (slice(None), modelclass)], c="red")

modelclass = "3d"
axes[2].fill_between(
    Y1.loc[(slice(None), modelclass), :].index.get_level_values("layer"),
    Y1.loc[(slice(None), modelclass), (slice(None), "min")].values.flat,
    Y1.loc[(slice(None), modelclass), (slice(None), "max")].values.flat,
)
axes[2].plot(-Y2.loc[:, (slice(None), modelclass)], c="red")

## predictor RDM (semantic-2d-3d)

Extend [model rdm](#model-rdm) to contain all models to use as predictor. Fill values for models not belonging to any class with zeros.

In [ ]:
predictor_rdm_1 = pd.DataFrame(0, index=_models, columns=_models)
for c1, c2 in combinations_with_replacement(_models, 2):
    if c1 in model_rdm.index and c2 in model_rdm.index:
        predictor_rdm_1.loc[c1, c2] = 1 - model_rdm.loc[c1, c2]
        predictor_rdm_1.loc[c2, c1] = 1 - model_rdm.loc[c2, c1]
    else:
        predictor_rdm_1.loc[c1, c2] = 0
        predictor_rdm_1.loc[c2, c1] = 0

# Predictor 2:  integration profile across layers

RDM of RDM's that correlate integration ratings of each different layers inside each network.

[Finished predictor RDM](#predictor-rdm-layer-layer-similarity-inside-networks)

TODO: this essentially the same thing as absoloute correlation differences alone ?
copy code for each models layerXlayer RDM
correlate correlations using daniels code

## version 1: layer X layer RDM for each network

In [ ]:
df1 = dfi.loc[(slice(None), slice(None), _scale, slice(None)), :].droplevel(["scale"])

In [ ]:
v1_rdms_p1, v1_rdms_p2, v1_rdms_oa = {}, {}, {}
for mo, df_mo in df1.groupby("model"):
    v1_rdms_p1[mo] = calculate_rdm(df_mo.loc[(slice(None), "places1"), :], "spearman")
    v1_rdms_p2[mo] = calculate_rdm(df_mo.loc[(slice(None), "places2"), :], "spearman")
    v1_rdms_oa[mo] = calculate_rdm(df_mo.loc[(slice(None), "oasis"), :], "spearman")

In [ ]:
PATH = os.path.join(PATH_RESULTS, "layer profile", "version 1")
with open(os.path.join(PATH, "rmds places1.pkl"), "wb") as f:
    pickle.dump(v1_rdms_p1, f)

with open(os.path.join(PATH, "rmds places2.pkl"), "wb") as f:
    pickle.dump(v1_rdms_p2, f)

with open(os.path.join(PATH, "rmds oasis.pkl"), "wb") as f:
    pickle.dump(v1_rdms_oa, f)

In [ ]:
PATH = os.path.join(PATH_RESULTS, "layer profile", "version 1")
with open(os.path.join(PATH, "rmds places1.pkl"), "rb") as f:
    v1_rdms_p1 = pickle.load(f)

with open(os.path.join(PATH, "rmds places2.pkl"), "rb") as f:
    v1_rdms_p2 = pickle.load(f)

with open(os.path.join(PATH, "rmds oasis.pkl"), "rb") as f:
    v1_rdms_oa = pickle.load(f)

## version 2: integration in best layer

ordering of images by integration in best predicting layer

"what is integrated", alternatively average of correlation between in each layer, howevery layers may not correspond to each other, therefore best predicting layer is more general <br> <br>

Interpretation: The differences in absolout values correspond to how similar the "integration mechanism" in both networks are.<br> If we assume that beauty perception depends on a specific stage of processing and not the whole processing stream, then the best predicting layer of a network can be interpreted as the point, where the network best mimics the aspects of the processing that determine beauty. <br> 

If the a similar The value in Is there a single or are there different ways of predicting beauty ?`

In [ ]:
# best layer per model
bestlayers = dfibc_m.groupby("model").idxmax().ibcorr
df_bestpredicting_integration = (
    dfi_m.unstack("img").loc[bestlayers].droplevel("layer").T.droplevel(0)
)

## predictor RDM

In [ ]:
# version 1: correlate network RDMs
# RDMs into columns
# throw out zeros on diagonal to avoid skewing correlation (standard RSA procedure)
RDMs_places1 = pd.DataFrame(columns=_models)

for network_name, rdm in v1_rdms_p1.items():
    # mark diagonal values (all zeros)for removal
    np.fill_diagonal(rdm.values, np.nan)
    RDMs_places1.loc[:, network_name] = rdm.values.flatten()

# removed marked diagonal values
RDMs_places1 = RDMs_places1.dropna()

print("Should be (2353, 15)")
print(RDMs_places1.shape)

predictor_rdm_2 = calculate_rdm(RDMs_places1)

In [ ]:
# version 2: only best layer
predictor_rdm_2 = calculate_rdm(df_bestpredicting_integration, "spearman")

# Predictor 3: spatial integration

"how"

"where" or alternatively "what",  which is the same because its spatial integration. Check for correlation between the what (represented by the integration ratings).

DONE IN SEPERATE NOTEBOOK

In [ ]:
# # import results
# with open(os.path.join(PATH_RESULTS, "spatial integration", "study1.pkl"), "rb") as f:
#     d1 = pickle.load(f)

# with open(os.path.join(PATH_RESULTS, "spatial integration", "study2.pkl"), "rb") as f:
#     d2 = pickle.load(f)

# with open(os.path.join(PATH_RESULTS, "spatial integration", "study3.pkl"), "rb") as f:
#     d3 = pickle.load(f)

# with open(os.path.join(PATH_RESULTS, "spatial integration", "study4.pkl"), "rb") as f:
#     d4 = pickle.load(f)


# rdm_study1 = calculate_rdm(flatten_concat(d1))
# rdm_study2 = calculate_rdm(flatten_concat(d2))
# rdm_study3 = calculate_rdm(flatten_concat(d3))
# rdm_study4 = calculate_rdm(flatten_concat(d4))

In [ ]:
_study = "short presentation"
_scale = "scale4"


rdms = {}

for mo in _models:
    rdm = pd.read_csv(
        os.path.join(
            PATH_RESULTS,
            "spatial integration",
            _scale,
            _study,
            mo + ".csv",
        ),
        header=None,
    )
    rdm = rdm.iloc[1:, 1:].reset_index(drop=True).T.reset_index(drop=True).T

    rdms[mo] = rdm

In [ ]:
sns.heatmap(spatial_rdm, xticklabels=spatial_rdm.index, yticklabels=spatial_rdm.columns)

## integration is localized

In [ ]:
# visualize node score distribution

In [ ]:
# visualize within layer heatmaps

# exemplars

In [ ]:
# spatial correlation per image per net, correlate these netXnet
# test if integration scores are still correlating to beauty

## predctor RDM

In [ ]:
predictor_rdm_3 = spatial_rdm

# Predictors-Target analysis
do for each study and each scale, to check if there is some consistency in which factors always comes out on top



### functions

In [ ]:
def calculate_predictors_target_rdms(
    path_integration, path_ibcorr, _models, _study, _scale, _layers
):
    _dataset = studyname2datasetname(_study)
    blocking = None

    dfi = load_integration(path_integration)
    dfibc = load_ibcorr(path_ibcorr)

    dfi = dfi.sort_index().loc[(_models, _dataset, _scale, slice(None), _layers)]
    dfi = dfi.droplevel(["dataset", "scale"])

    dfibc = dfibc.sort_index().loc[(_models, _study, _scale, _layers)]
    dfibc = dfibc.droplevel(["study", "scale"])

    # --- target ---
    target_rdm = pd.DataFrame(0, index=_models, columns=_models)
    df = dfibc.unstack("layer")
    for c1, c2 in combinations(_models, 2):
        target_rdm.loc[c1, c2] = (df.loc[c1] - df.loc[c2]).abs().sum()
        target_rdm.loc[c2, c1] = (df.loc[c1] - df.loc[c2]).abs().sum()

    # --- predictor 1 ---
    model_rdm = pd.DataFrame(
        np.full((len(NETS_ALL), len(NETS_ALL)), np.nan),
        columns=NETS_ALL,
        index=NETS_ALL,
    )
    for combi in combinations_with_replacement(NETS_ALL, 2):
        if (
            combi in combinations_with_replacement(NETS_SEMANTIC, 2)
            or combi in combinations_with_replacement(NETS_2D, 2)
            or combi in combinations_with_replacement(NETS_3D, 2)
        ):
            model_rdm.loc[combi] = 1
            model_rdm.loc[tuple(reversed(combi))] = 1
        else:
            model_rdm.loc[combi] = 0
            model_rdm.loc[tuple(reversed(combi))] = 0

    predictor_rdm_1 = pd.DataFrame(0, index=_models, columns=_models)
    for c1, c2 in combinations_with_replacement(_models, 2):
        if c1 in model_rdm.index and c2 in model_rdm.index:
            predictor_rdm_1.loc[c1, c2] = 1 - model_rdm.loc[c1, c2]
            predictor_rdm_1.loc[c2, c1] = 1 - model_rdm.loc[c2, c1]
        else:
            predictor_rdm_1.loc[c1, c2] = 0
            predictor_rdm_1.loc[c2, c1] = 0
    
    # --- predictor 2---
    bestlayers = dfibc.groupby("model").idxmax().ibcorr
    df_bestpredicting_integration = (
        dfi.unstack("img").loc[bestlayers].droplevel("layer").T.droplevel(0)
    )
    predictor_rdm_2 = calculate_rdm(df_bestpredicting_integration, "spearman")

    # ---predictor 3---
    try:
        rdms = {}
        for mo in _models:
            rdm = pd.read_csv(
                os.path.join(
                    PATH_RESULTS,
                    "spatial integration",
                    _scale,
                    _study,
                    mo + ".csv",
                ),
                header=None,
            )
            rdm = rdm.iloc[1:, 1:].reset_index(drop=True).T.reset_index(drop=True).T
            rdms[mo] = rdm
        predictor_rdm_3 = calculate_rdm(flatten_concat_rdms(rdms))
    except Exception as e:
        print("Spatial analysis not available for these parameters, returing emptry predictor 3")
        predictor_rdm_3 = None

    return target_rdm, predictor_rdm_1, predictor_rdm_2, predictor_rdm_3

In [ ]:
def correlate_rdms(rdm1, rdm2, correlation="pearson"):

    mask = np.triu(np.ones_like(rdm1.values).astype(np.bool_), k=1)
    if correlation == "pearson":
        return pearsonr(rdm1.values[mask], rdm2.values[mask])

    if correlation == "spearman":
        return spearmanr(rdm1.values[mask], rdm2.values[mask])

    raise ValueError(
        "corrrelate_rdm muss für correlation pearson oder spearman bekommen"
    )

In [ ]:
def rdm2vec(rdm):
    mask = np.triu(np.ones_like(rdm.values).astype(np.bool_), k=1)
    return rdm.values[mask]

##

In [ ]:
print(
    "blocking:",
    _blocking,
    " | ",
    len(_models),
    " | ",
    _study,
    " | ",
    _scale,
    " | ",
    _layers,
)

###
old

In [ ]:
# predictor 1 alone
print(
    "blocking:",
    _blocking,
    " | ",
    len(_models),
    " | ",
    _study,
    " | ",
    _scale,
    " | ",
    _layers,
)
correlate_rdms(target_rdm, predictor_rdm_1, correlation="pearson")

In [ ]:
print(
    "blocking:",
    _blocking,
    " | ",
    len(_models),
    " | ",
    _study,
    " | ",
    _scale,
    " | ",
    _layers,
)
correlate_rdms(target_rdm, predictor_rdm_1, correlation="pearson")

In [ ]:
print(
    "blocking:",
    _blocking,
    " | ",
    len(_models),
    " | ",
    _study,
    " | ",
    _scale,
    " | ",
    _layers,
)
correlate_rdms(target_rdm, predictor_rdm_1, correlation="pearson")

### correlation

In [ ]:
target, pred1, pred2, pred3 = calculate_predictors_target_rdms(PATH_INTEGRATION, PATH_IBCORR, MODEL_NAMES, "short presentation", "scale4", slice(None))
correlate_rdms(target, pred1), correlate_rdms(target, pred2), correlate_rdms(target, pred3)

In [ ]:
target, pred1, pred2, pred3 = calculate_predictors_target_rdms(PATH_INTEGRATION, PATH_IBCORR, _models, "short presentation", "scale4", slice(None))
correlate_rdms(target, pred1), correlate_rdms(target, pred2), correlate_rdms(target, pred3)

In [ ]:
target, pred1, pred2, pred3 = calculate_predictors_target_rdms(PATH_INTEGRATION_MAXPOOL, PATH_IBCORR_MAXPOOL, MODEL_NAMES, "short presentation", "scale4", slice(None))
correlate_rdms(target, pred1), correlate_rdms(target, pred2), correlate_rdms(target, pred3)

In [ ]:
target, pred1, pred2, pred3 = calculate_predictors_target_rdms(PATH_INTEGRATION_MAXPOOL, PATH_IBCORR_MAXPOOL, _models, "short presentation", "scale4", slice(None))
correlate_rdms(target, pred1), correlate_rdms(target, pred2), correlate_rdms(target, pred3)

In [ ]:
target, pred1, pred2, pred3 = calculate_predictors_target_rdms(PATH_INTEGRATION_AVERAGE, PATH_IBCORR_AVERAGE, MODEL_NAMES, "short presentation", "scale4", slice(6,14+1))
correlate_rdms(target, pred1), correlate_rdms(target, pred2), correlate_rdms(target, pred3)

In [ ]:
target, pred1, pred2, pred3 = calculate_predictors_target_rdms(PATH_INTEGRATION_MAXPOOL, PATH_IBCORR_MAXPOOL, _models, "short presentation", "scale4", slice(6,14+1))
correlate_rdms(target, pred1), correlate_rdms(target, pred2), correlate_rdms(target, pred3)

### variance partitioning

In [ ]:
# function for variance partitioning
def predictors_r2(predictors, target):
    predictors = sm.add_constant(predictors)
    model = sm.OLS(target, predictors)
    results = model.fit()
    return results.rsquared    

In [ ]:
t, p1, p2, p3 = rdm2vec(target), rdm2vec(pred1), rdm2vec(pred2), rdm2vec(pred3)

In [ ]:
predictors_r2(p1, t), predictors_r2(p2, t), predictors_r2(p3, t)

In [ ]:
predictors_r2(np.stack((p1, p3), axis=1), t) - predictors_r2(p3, t)

In [ ]:
# plot full venn diagram (with 2 or 3 predictors)

### Visualize predictor 3

In [ ]:
spearmanr(t, p3)

In [ ]:
plt.scatter(t, p3, s=10, color="magenta")

In [ ]:
sns.kdeplot(pd.DataFrame((t, p3)).T, x=0, y=1, fill=True,levels=100)
plt.scatter(t, p3, s=10, color="magenta")
plt.xlim(0,t.max())
plt.ylim(0,p1.max())

In [ ]:
plt.scatter(t.argsort(), p3.argsort(), s=10, color="magenta")

In [ ]:
sns.kdeplot(pd.DataFrame((t.argsort(), p3.argsort())).T, x=0, y=1, fill=True,levels=3)
plt.scatter(t.argsort(), p3.argsort(), s=10, color="magenta")
plt.xlim(0,t.size)
plt.ylim(0,t.size)

###